In [1]:
from EEGDatasets import HMSDataset
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'torch._C'

In [10]:
hmsdataset = HMSDataset(train_grouped_eeg, parquet_file)
dataloader = DataLoader(hmsdataset, batch_size=2)

In [4]:
import pandas as pd
target_p = [
    'seizure_vote', 
    'lpd_vote', 
    'gpd_vote', 
    'lrda_vote', 
    'grda_vote', 
    'other_vote'
]

In [5]:
train_file = '/data/hms/train.csv'
parquet_file = '/data/hms/train_eegs/'

In [6]:
train = pd.read_csv(train_file)
train['total_evaluators'] = train[target_p].sum(axis=1)


train_grouped_eeg = train.groupby('eeg_id').agg({
    'eeg_sub_id': 'count',  # 计算每个 eeg_id 对应的行数
    'eeg_label_offset_seconds': 'first',
    'spectrogram_id': 'first',
    'spectrogram_sub_id': 'first',
    'spectrogram_label_offset_seconds': 'first',
    'label_id': 'first',
    'patient_id': 'first',
    'expert_consensus': 'first',
    'seizure_vote': 'sum',
    'lpd_vote': 'sum',
    'gpd_vote': 'sum',
    'lrda_vote': 'sum',
    'grda_vote': 'sum',
    'other_vote': 'sum',
    'total_evaluators': 'sum'
}).reset_index()
    
train_grouped_eeg['consensus'] = train_grouped_eeg[target_p].max(axis=1)

# 计算每一列的比例
train_grouped_eeg[target_p] = train_grouped_eeg[target_p].div(
    train_grouped_eeg['total_evaluators'], 
    axis=0
)
    
train_grouped_eeg

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,total_evaluators,consensus
0,568657,4,0.0,789577333,0,0.0,1825637311,20654,Other,0.0,0.000000,0.25,0.000000,0.166667,0.583333,48,28
1,582999,11,0.0,1552638400,0,0.0,1722186807,20230,LPD,0.0,0.857143,0.00,0.071429,0.000000,0.071429,154,132
2,642382,2,0.0,14960202,12,1008.0,3254468733,5955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000,2,2
3,751790,1,0.0,618728447,4,908.0,2898467035,38549,GPD,0.0,0.000000,1.00,0.000000,0.000000,0.000000,1,1
4,778705,1,0.0,52296320,0,0.0,3255875127,40955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1,0.0,1188113564,0,0.0,447244163,16610,GRDA,0.0,0.000000,0.00,0.000000,0.500000,0.500000,2,1
17085,4293843368,1,0.0,1549502620,0,0.0,1618953053,15065,GRDA,0.0,0.000000,0.00,0.000000,0.500000,0.500000,2,1
17086,4294455489,1,0.0,2105480289,0,0.0,469526364,56,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000,1,1
17087,4294858825,5,0.0,657299228,0,0.0,561576493,4312,Other,0.0,0.000000,0.00,0.000000,0.066667,0.933333,75,70


In [11]:
hmsdataset = HMSDataset(train_grouped_eeg, parquet_file)
dataloader = DataLoader(hmsdataset, batch_size=2)

NameError: name 'nn' is not defined

In [ ]:
class CustomCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CustomCrossEntropyLoss, self).__init__()
        
    def forward(self, y_hat, y):
        y_hat = F.log_softmax(y_hat, dim=-1)
        return -torch.sum(y * torch.log(y_hat + 1e-10)) / y.shape[0]

In [44]:
from datasets import Dataset

In [54]:
dataset = Dataset.from_list(EEGDataset(df, eeg))

In [ ]:
dataset = EEGDataset(df, eeg)

In [ ]:
from transformers import PatchTSTConfig, PatchTSTForClassification

# classification task with two input channel2 and 3 classes
config = PatchTSTConfig(
    num_input_channels=8,
    num_targets=6,
    context_length=512,
    patch_length=12,
    stride=12,
    use_cls_token=True,
)
model = PatchTSTForClassification(config=config)

# during inference, one only provides past values
# past_values = torch.randn(20, 512, 2)
# outputs = model(past_values=past_values)
# labels = outputs.prediction_logits

In [ ]:
import torch
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=128, num_workers=8, shuffle=True)

In [40]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=small_eval_dataset,
    # compute_metrics=compute_metrics,
)

In [53]:
model.train()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1):
    # Training
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()


X


TypeError: ones_like(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
trainer.save_model("test_trainer")